In [1]:
from src.Wrapper import LlamaWrapper

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
llama = LlamaWrapper(model_id=model_id)


/data/miniconda3/envs/torch21/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/miniconda3/envs/torch21/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.48it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
title_list = [
    "Dream Theater Metropolis Pt. 1 (Live At Luna Park DVD)", 
    "Plini & David Maxim Micic LIVE @ Vh1 Supersonic 2018 (full set)",
    "(Måneskin) Beggin' - Fingerstyle Guitar Cover | Josephine Alexandra",
    "Owane - Rock Is Too Heavy",
    "Jungle - Tash Sultana - Tutorial - Guitar Loop Cover - Tabs Available",
    "How to Play Beat It Solo - Eddie Van Halen Michael Jackson"
    ]


# Artist and Title

#### Option 1) manually with full list --> unstructured output

In [3]:
system_prompt = "For the following list of video titles, return dictionaries per element with artists and titles. The keys are the utterance and the values either ARTIST or TITLE."

temperature = 0.1
top_p = 0.1

output = llama.prompt_list(system_prompt, title_list, temperature=temperature, top_p=top_p)
output


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Here is the list of dictionaries:\n\n[\n    {\'utterance\': \'Dream Theater Metropolis Pt. 1 (Live At Luna Park DVD)\', \'ARTIST\': \'Dream Theater\', \'TITLE\': \'Metropolis Pt. 1 (Live At Luna Park DVD)\'},\n    {\'utterance\': \'Plini & David Maxim Micic LIVE @ Vh1 Supersonic 2018 (full set)\', \'ARTIST\': \'Plini & David Maxim Micic\', \'TITLE\': \'LIVE @ Vh1 Supersonic 2018 (full set)\'},\n    {\'utterance\': "(Måneskin) Beggin\' - Fingerstyle Guitar Cover | Josephine Alexandra", \'ARTIST\': \'Måneskin\', \'TITLE\': \'Beggin\\\'\'},\n    {\'utterance\': \'Owane - Rock Is Too Heavy\', \'ARTIST\': \'Owane\', \'TITLE\': \'Rock Is Too Heavy\'},\n    {\'utterance\': \'Jungle - Tash Sultana - Tutorial - Guitar Loop Cover - Tabs Available\', \'ARTIST\': \'Jungle & Tash Sultana\', \'TITLE\': \'Tutorial - Guitar Loop Cover\'},\n    {\'utterance\': \'How to Play Beat It Solo - Eddie Van Halen'

#### Option 2) with jsonformer --> structured but only single json.

In [9]:
json_schema = {
    "type": "object",
    "properties": {
        "artist": {"type": "string"},
        "title": {"type": "string"}
        }
    }

for title in title_list:  
    prompt = f"For the following list of video titles, parse the respective musical artists and song titles into the following schema. {title}"
    output = llama.prompt_to_json(prompt, json_schema, 0.1)
    print(output)



{'artist': 'Dream Theater', 'title': 'Metropolis Pt. 1'}
{'artist': 'Plini', 'title': 'LIVE @ Vh1 Supersonic 201'}
{'artist': 'Måneskin', 'title': "Beggin'"}
{'artist': 'Owane', 'title': 'Rock Is Too Heavy'}
{'artist': 'Tash Sultana', 'title': 'Jungle'}
{'artist': 'Eddie Van Halen', 'title': 'Beat It Solo'}


# NER

In [3]:
system_prompt = "Return a list with dictionaries per user prompt which contain the entity types as keys mapping to named entities in the text. Here is the list: "

temperature = 0.1
top_p = 0.1

output = llama.prompt_list(system_prompt, user_prompt, temperature=temperature, top_p=top_p)
output


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Here is the list with dictionaries per user prompt which contain the entity types as keys mapping to named entities in the text:\n\n```\n[\n    {\n        'ORGANIZATION': ['Dream Theater', 'Luna Park', 'Vh1', 'Supersonic'],\n        'PERSON': ['Plini', 'David Maxim Micic']\n    },\n    {\n        'ORGANIZATION': ['Plini', 'David Maxim Micic', 'Vh1', 'Supersonic'],\n        'WORK_OF_ART': ['Metropolis Pt. 1', 'Live At Luna Park DVD']\n    }\n]\n```"